<a href="https://colab.research.google.com/github/Rafael-Salomao/Criando_Vari-veis_Preditivas_Pyspark/blob/main/Criando-Vari%C3%A1veis-Preditivas-Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Criando Variáveis Preditivas - Pyspark.**

## **1 - Introdução**

**1. Contextualização:**

Na execução de projetos de Ciência de Dados, enfrentamos um dos principais desafios: extrair o máximo de informações valiosas dos conjuntos de dados disponíveis. Para atender a esse desafio, o Engenheiro de Dados deve empregar estratégias que aprimorem esses dados. Uma das abordagens fundamentais para otimizar o desempenho de modelos estatísticos é a criação de variáveis preditivas.

**2. Objetivo do Projeto:**

O objetivo central deste projeto é demonstrar como podemos enriquecer significativamente a quantidade de informações disponíveis, aproveitando o cruzamento de dados transacionais. Isso é alcançado por meio de análises estatísticas que nos permitem calcular médias, valores máximos e mínimos de transações realizadas por indivíduos em diferentes categorias ao longo do tempo. Essas informações geradas se revelam incrivelmente valiosas na resolução de diversos problemas de crédito, tais como:

* Avaliação de Risco de Inadimplência
* Determinação de Limites de Crédito
* Detecção de Fraude

**3. Como Foi Feito:**

Para atingir esse objetivo, foi realizado um processo parcial de ETL nos dados da competição "Home Credit Default Risk" disponível no Kaggle. A metodologia empregada envolve o uso do poder de processamento do PySpark, conhecido por oferecer recursos escaláveis e eficazes no processamento de grandes volumes de dados.

Isso é alcançado por meio de análises estatísticas que nos permitem calcular médias, valores máximos e mínimos de transações realizadas por indivíduos em diferentes categorias ao longo do tempo.

Ao explorar essa abordagem para enriquecer dados transacionais, esta solução contribui para soluções mais eficazes e precisas em modelos preditivos relacionados ao crédito, proporcionando benefícios significativos tanto para as instituições financeiras quanto para os consumidores.

## **2- Importar Pyspark**

In [ ]:
!pip install Pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Exercicio 2 Aula 17 Pyspark').getOrCreate()

In [ ]:
spark

## **3 - Importar Metadados**

- Para entendimento da tabela.

In [ ]:
# Selecionar caminho do arquivo
caminho_descricao_colunas = '/content/HomeCredit_columns_description.csv'

# Ler o arquivo de descrição das colunas
df_descricao_colunas = spark.read.csv(caminho_descricao_colunas, header=True)

# Mostrar caminho de descrição de colunas
df_descricao_colunas.show(truncate=False)

+---+----------------------------+--------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------+
|_c0|Table                       |Row                       |Description                                                                                                                                                                                |Special                              |
+---+----------------------------+--------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------+
|1  |application_{train|test}.csv|SK_ID_CURR                |ID of loan in our sample                                                   

## **4 - Importar tabela 'bureau'.**

- A tabela de bureau tem dados de emprestimos anteriores que o cliente pegou de outras instituições que foram reportadas para os bureaus de créditos.

- Essa tabela apresenta uma linha por cada emprestimo feito pelo usuario

### **4.1 - Mostrar primeiras linhas e colunas**

In [ ]:
# Selecionar caminho do arquivo
caminho_bureau = '/content/bureau.csv'

# Ler o arquivo csv
df_bureau_000 = spark.read.csv(caminho_bureau, header=True)

# Imprimir o arquivo credit card balance
df_bureau_000.show()

+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|    CREDIT_TYPE|DAYS_CREDIT_UPDATE|AMT_ANNUITY|
+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|    215354|     5714462|       Closed|     currency 1|       -497|                 0|             -153.0|           -153

### **4.2 - Mostrar tipo das colunas**

In [ ]:
df_bureau_000.printSchema()

root
 |-- SK_ID_CURR: string (nullable = true)
 |-- SK_ID_BUREAU: string (nullable = true)
 |-- CREDIT_ACTIVE: string (nullable = true)
 |-- CREDIT_CURRENCY: string (nullable = true)
 |-- DAYS_CREDIT: string (nullable = true)
 |-- CREDIT_DAY_OVERDUE: string (nullable = true)
 |-- DAYS_CREDIT_ENDDATE: string (nullable = true)
 |-- DAYS_ENDDATE_FACT: string (nullable = true)
 |-- AMT_CREDIT_MAX_OVERDUE: string (nullable = true)
 |-- CNT_CREDIT_PROLONG: string (nullable = true)
 |-- AMT_CREDIT_SUM: string (nullable = true)
 |-- AMT_CREDIT_SUM_DEBT: string (nullable = true)
 |-- AMT_CREDIT_SUM_LIMIT: string (nullable = true)
 |-- AMT_CREDIT_SUM_OVERDUE: string (nullable = true)
 |-- CREDIT_TYPE: string (nullable = true)
 |-- DAYS_CREDIT_UPDATE: string (nullable = true)
 |-- AMT_ANNUITY: string (nullable = true)



## **5 - Criação de variáveis preditivas**

O objetivo desta etapa é criar novas variáveis preditivas a partir das informações contidas na tabela 'bureau', a fim de enriquecer a tabela de treino. A tabela de treino possui uma linha para cada ID de usuário, enquanto a tabela 'bureau' contém informações de várias transações realizadas por um usuário. Cada usuário tem várias linhas correspondentes a cada transação efetuada.

O intuito deste processo ETL é gerar novas variáveis explicativas com base nessas transações, consolidando as diferentes linhas em uma única linha para cada usuário. Isso permitirá enriquecer as informações de cada usuário na tabela de treino e aprimorar o poder preditivo dos modelos estatísticos desenvolvidos posteriormente pelo Cientista de Dados.

A criação das novas variáveis será realizada utilizando o Spark SQL. Inicialmente, faremos um exemplo dessa criação de variável. Em seguida, apresentaremos uma forma de automatizar e dimensionar essa prática, que pode fazer a diferença no desempenho de um modelo. Consequentemente, terá um impacto positivo na receita gerada por esse modelo em um negócio.








### **5.1 - Agrupando Variáveis Categóricas**

Considerando a tabela de 'bureau', que estamos trabalhando, para criar novas variáveis a partir das informações contempladas nas diferentes linhas, é possível agrupar essas variáveis categóricas.

Ou seja, considerando um único clientes, que conta com diferentes registros, é possível contar a quantidade de transações que constam como 'Active' na coluna 'Credit_Active', por exemplo, ou quantas estão 'Sold', 'Closed', 'Bad Debt'...

Neste caso, para cada elemento desta variável categórica, é criado uma nova coluna referente ao cliente em questão, especificando a quantidade de cada situação de seu credit ativo.

Para elucidar este exemplo, é considerado a criação das variáveis apenas para o usuário com "SK_ID_CURR = '215354'", conforme é possível verificar no exemplo abaixo:

In [ ]:
df_bureau_000.createOrReplaceTempView('df_view')
df_bureau_001 = spark.sql('''
SELECT
  SK_ID_CURR,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN 1 ELSE 0 END) as sum_credit_active_bad_debt,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN 1 ELSE 0 END) as sum_credit_active_sold,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Active' THEN 1 ELSE 0 END) as sum_credit_active_active,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN 1 ELSE 0 END) as sum_credit_active_closed
FROM
  df_view
WHERE
  SK_ID_CURR = '215354'
GROUP BY
  SK_ID_CURR
''')
df_bureau_001.show()

+----------+--------------------------+----------------------+------------------------+------------------------+
|SK_ID_CURR|sum_credit_active_bad_debt|sum_credit_active_sold|sum_credit_active_active|sum_credit_active_closed|
+----------+--------------------------+----------------------+------------------------+------------------------+
|    215354|                         0|                     0|                       6|                       1|
+----------+--------------------------+----------------------+------------------------+------------------------+



In [ ]:
# +----------+--------------------------+----------------------+------------------------+------------------------+
# |SK_ID_CURR|sum_credit_active_bad_debt|sum_credit_active_sold|sum_credit_active_active|sum_credit_active_closed|
# +----------+--------------------------+----------------------+------------------------+------------------------+
# |    215354|                         0|                     0|                       6|                       1|
# +----------+--------------------------+----------------------+------------------------+------------------------+

### **5.2 - Agrupando Variáveis Categóricas junto com Variáveis Numéricas**

Para enriquecer ainda mais esta criação de variáveis, é possível combinar a criação de novas variáveis, combinando as variáveis categóricas com as variáveis numéricas. Desta forma, é possível verificar "a soma total de crédito do cliente quando seu crédito está ativo", por exemplo.

No exemplo abaixo, considerando o mesmo usuário visto anteriormente, é possível verificar as novas variáveis que serão criadas.


In [ ]:
df_bureau_002 = spark.sql('''
SELECT
  SK_ID_CURR,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_active_bad_debt,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_active_sold,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Active' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_active_active,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_active_closed
FROM
  df_view
WHERE
  SK_ID_CURR = '215354'
GROUP BY
  SK_ID_CURR
''')
df_bureau_002.show()

+----------+-------------------------------------+---------------------------------+-----------------------------------+-----------------------------------+
|SK_ID_CURR|amt_credit_sum_credit_active_bad_debt|amt_credit_sum_credit_active_sold|amt_credit_sum_credit_active_active|amt_credit_sum_credit_active_closed|
+----------+-------------------------------------+---------------------------------+-----------------------------------+-----------------------------------+
|    215354|                                  0.0|                              0.0|                          3701427.3|                            91323.0|
+----------+-------------------------------------+---------------------------------+-----------------------------------+-----------------------------------+



In [ ]:
# +----------+-------------------------------------+---------------------------------+-----------------------------------+-----------------------------------+
# |SK_ID_CURR|amt_credit_sum_credit_active_bad_debt|amt_credit_sum_credit_active_sold|amt_credit_sum_credit_active_active|amt_credit_sum_credit_active_closed|
# +----------+-------------------------------------+---------------------------------+-----------------------------------+-----------------------------------+
# |    215354|                                  0.0|                              0.0|                          3701427.3|                            91323.0|
# +----------+-------------------------------------+---------------------------------+-----------------------------------+-----------------------------------+

### **5.3 - Escalando a Criação de Variáveis.**

Uma vez que vimos a criação de novas variáveis mostradas anteriormente, podemos considerar que é possível criar ainda mais variáveis considerando o restante das variáveis encontradas na tabela 'bureau'.

Para isso, podemos considerar que é possível fazer diferentes agregações para cada uma das variáveis categóricas. Tais agregações que ao serem consideradas com as variáveis numéricas, podem enriquecer ainda mais a informações de cada usuário, representando um alto ganho de desempenho nos modelos preditivos e gerando mais valor ao negócio.

Sendo assim, a partir da estrutura de repetições do python, pode-se automatizar esta criação de variáveis para auxiliar na velocidade e eficiencia desta prática.

Tendo isso em vista, é preciso elencar as agregações, variáveis categóricas e variáveis numéricas que serão utilizadas, assim como também é preciso considerar os domínios das variáveis categóricas que serão consideradas na estrutura de repetiçãos.

#### **5.3.1 - Elencar Agregações, Variáveis Categóricas, Variáveis Numéricas**

In [ ]:
agg = ['SUM', 'MAX', 'MIN', 'AVG']

var_categoricas = ['CREDIT_ACTIVE', 'CREDIT_TYPE', 'CREDIT_CURRENCY']

var_numericas = ['AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM', 'AMT_CREDIT_MAX_OVERDUE']

#### **5.3.2 - Elencar Domínios das Variáveis Categóricas**

In [ ]:
df_bureau_003 = spark.sql('''
SELECT
  CREDIT_ACTIVE,
  COUNT(SK_ID_BUREAU)
FROM
  df_view
GROUP BY
  CREDIT_ACTIVE
''').collect()

dominios_categoricas_credit_active = [row.CREDIT_ACTIVE for row in df_bureau_003]

# Print the unique values list
print(dominios_categoricas_credit_active)

['Bad debt', 'Sold', 'Active', 'Closed']


In [ ]:
df_bureau_004 = spark.sql('''
SELECT
  CREDIT_TYPE,
  COUNT(SK_ID_BUREAU)
FROM
  df_view
GROUP BY
  CREDIT_TYPE
''').collect()

dominios_categoricas_credit_type = [row.CREDIT_TYPE for row in df_bureau_004]

# Print the unique values list
print(dominios_categoricas_credit_type)

['Microloan', 'Consumer credit', 'Another type of loan', 'Mortgage', 'Loan for working capital replenishment', 'Car loan', 'Real estate loan', 'Unknown type of loan', 'Loan for business development', 'Credit card']


In [ ]:
df_bureau_005 = spark.sql('''
SELECT
  CREDIT_CURRENCY,
  COUNT(SK_ID_BUREAU)
FROM
  df_view
GROUP BY
  CREDIT_CURRENCY
''').collect()

dominios_categoricas_credit_currency = [row.CREDIT_CURRENCY for row in df_bureau_005]

# Print the unique values list
print(dominios_categoricas_credit_currency)

['currency 2', 'currency 1', 'currency 4', 'currency 3']


#### **5.3.3 - Montar Estrutura de Repetição**

A estrutura de repetição é feita considerando as variáveis elencadas anteriormente, assim como, considerando a estrutura do código em spark sql, para a criação do código de agregação e criação das variáveis.

Com isso, basta incluir o resultado encontrado nesta estrutura, no código spark sql para obter a criação de inúmeras novas variáveis e informações que estavam "escondidas" nas base da dados inicial.

Este exemplo foi capaz de gerar mais de 350 variáveis preditivas que são capazes de gerar muito valor e com certeza elevar o nível do resultado de modelos estatísticos para outro patamar.

In [ ]:
for ag in agg:
  for var_cat in var_categoricas:
    if var_cat == 'CREDIT_ACTIVE':
      for dom_cat_credit_active in dominios_categoricas_credit_active:
        for var_numerica in var_numericas:
          print( ag + '(CASE WHEN ' + var_cat + " = '" + dom_cat_credit_active + "' THEN " + var_numerica + ' ELSE 0 END) as ' + var_numerica.lower() + '_' + var_cat.lower() + '_' + dom_cat_credit_active.lower().replace(' ', '_') + ',')
    if var_cat == 'CREDIT_TYPE':
      for dom_cat_credit_type in dominios_categoricas_credit_type:
        for var_numerica in var_numericas:
          print( ag + '(CASE WHEN ' + var_cat + " = '" + dom_cat_credit_type + "' THEN " + var_numerica + ' ELSE 0 END) as ' + var_numerica.lower() + '_' + var_cat.lower() + '_' + dom_cat_credit_type.lower().replace(' ', '_').replace('(','').replace(')','').replace('-','_') + ',')
    if var_cat == 'CREDIT_CURRENCY':
      for dom_cat_credit_currency in dominios_categoricas_credit_currency:
        for var_numerica in var_numericas:
          print( ag + '(CASE WHEN ' + var_cat + " = '" + dom_cat_credit_currency + "' THEN " + var_numerica + ' ELSE 0 END) as ' + var_numerica.lower() + '_' + var_cat.lower() + '_' + dom_cat_credit_currency.lower().replace(' ', '_') + ',')

SUM(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_active_bad_debt,
SUM(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_active_bad_debt,
SUM(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_active_bad_debt,
SUM(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_active_bad_debt,
SUM(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_active_sold,
SUM(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_active_sold,
SUM(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_active_sold,
SUM(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_active_sold,
SUM(CASE WHEN CREDIT_ACTIVE = 'Activ

In [ ]:
df_bureau_006 = spark.sql('''
SELECT
  SK_ID_CURR,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_active_bad_debt,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_active_bad_debt,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_active_bad_debt,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_active_bad_debt,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_active_sold,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_active_sold,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_active_sold,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_active_sold,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Active' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_active_active,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Active' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_active_active,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Active' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_active_active,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Active' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_active_active,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_active_closed,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_active_closed,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_active_closed,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_active_closed,
  SUM(CASE WHEN CREDIT_TYPE = 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_loan_for_the_purchase_of_equipment,
  SUM(CASE WHEN CREDIT_TYPE = 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_loan_for_the_purchase_of_equipment,
  SUM(CASE WHEN CREDIT_TYPE = 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_loan_for_the_purchase_of_equipment,
  SUM(CASE WHEN CREDIT_TYPE = 'Loan for the purchase of equipment' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_loan_for_the_purchase_of_equipment,
  SUM(CASE WHEN CREDIT_TYPE = 'Cash loan (non-earmarked)' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_cash_loan_non_earmarked,
  SUM(CASE WHEN CREDIT_TYPE = 'Cash loan (non-earmarked)' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_cash_loan_non_earmarked,
  SUM(CASE WHEN CREDIT_TYPE = 'Cash loan (non-earmarked)' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_cash_loan_non_earmarked,
  SUM(CASE WHEN CREDIT_TYPE = 'Cash loan (non-earmarked)' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_cash_loan_non_earmarked,
  SUM(CASE WHEN CREDIT_TYPE = 'Microloan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_microloan,
  SUM(CASE WHEN CREDIT_TYPE = 'Microloan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_microloan,
  SUM(CASE WHEN CREDIT_TYPE = 'Microloan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_microloan,
  SUM(CASE WHEN CREDIT_TYPE = 'Microloan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_microloan,
  SUM(CASE WHEN CREDIT_TYPE = 'Consumer credit' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_consumer_credit,
  SUM(CASE WHEN CREDIT_TYPE = 'Consumer credit' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_consumer_credit,
  SUM(CASE WHEN CREDIT_TYPE = 'Consumer credit' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_consumer_credit,
  SUM(CASE WHEN CREDIT_TYPE = 'Consumer credit' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_consumer_credit,
  SUM(CASE WHEN CREDIT_TYPE = 'Mobile operator loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_mobile_operator_loan,
  SUM(CASE WHEN CREDIT_TYPE = 'Mobile operator loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_mobile_operator_loan,
  SUM(CASE WHEN CREDIT_TYPE = 'Mobile operator loan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_mobile_operator_loan,
  SUM(CASE WHEN CREDIT_TYPE = 'Mobile operator loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_mobile_operator_loan,
  SUM(CASE WHEN CREDIT_TYPE = 'Another type of loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_another_type_of_loan,
  SUM(CASE WHEN CREDIT_TYPE = 'Another type of loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_another_type_of_loan,
  SUM(CASE WHEN CREDIT_TYPE = 'Another type of loan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_another_type_of_loan,
  SUM(CASE WHEN CREDIT_TYPE = 'Another type of loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_another_type_of_loan,
  SUM(CASE WHEN CREDIT_TYPE = 'Mortgage' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_mortgage,
  SUM(CASE WHEN CREDIT_TYPE = 'Mortgage' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_mortgage,
  SUM(CASE WHEN CREDIT_TYPE = 'Mortgage' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_mortgage,
  SUM(CASE WHEN CREDIT_TYPE = 'Mortgage' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_mortgage,
  SUM(CASE WHEN CREDIT_TYPE = 'Interbank credit' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_interbank_credit,
  SUM(CASE WHEN CREDIT_TYPE = 'Interbank credit' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_interbank_credit,
  SUM(CASE WHEN CREDIT_TYPE = 'Interbank credit' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_interbank_credit,
  SUM(CASE WHEN CREDIT_TYPE = 'Interbank credit' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_interbank_credit,
  SUM(CASE WHEN CREDIT_TYPE = 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_loan_for_working_capital_replenishment,
  SUM(CASE WHEN CREDIT_TYPE = 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_loan_for_working_capital_replenishment,
  SUM(CASE WHEN CREDIT_TYPE = 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_loan_for_working_capital_replenishment,
  SUM(CASE WHEN CREDIT_TYPE = 'Loan for working capital replenishment' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_loan_for_working_capital_replenishment,
  SUM(CASE WHEN CREDIT_TYPE = 'Car loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_car_loan,
  SUM(CASE WHEN CREDIT_TYPE = 'Car loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_car_loan,
  SUM(CASE WHEN CREDIT_TYPE = 'Car loan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_car_loan,
  SUM(CASE WHEN CREDIT_TYPE = 'Car loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_car_loan,
  SUM(CASE WHEN CREDIT_TYPE = 'Real estate loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_real_estate_loan,
  SUM(CASE WHEN CREDIT_TYPE = 'Real estate loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_real_estate_loan,
  SUM(CASE WHEN CREDIT_TYPE = 'Real estate loan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_real_estate_loan,
  SUM(CASE WHEN CREDIT_TYPE = 'Real estate loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_real_estate_loan,
  SUM(CASE WHEN CREDIT_TYPE = 'Unknown type of loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_unknown_type_of_loan,
  SUM(CASE WHEN CREDIT_TYPE = 'Unknown type of loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_unknown_type_of_loan,
  SUM(CASE WHEN CREDIT_TYPE = 'Unknown type of loan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_unknown_type_of_loan,
  SUM(CASE WHEN CREDIT_TYPE = 'Unknown type of loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_unknown_type_of_loan,
  SUM(CASE WHEN CREDIT_TYPE = 'Loan for business development' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_loan_for_business_development,
  SUM(CASE WHEN CREDIT_TYPE = 'Loan for business development' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_loan_for_business_development,
  SUM(CASE WHEN CREDIT_TYPE = 'Loan for business development' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_loan_for_business_development,
  SUM(CASE WHEN CREDIT_TYPE = 'Loan for business development' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_loan_for_business_development,
  SUM(CASE WHEN CREDIT_TYPE = 'Credit card' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_credit_card,
  SUM(CASE WHEN CREDIT_TYPE = 'Credit card' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_credit_card,
  SUM(CASE WHEN CREDIT_TYPE = 'Credit card' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_credit_card,
  SUM(CASE WHEN CREDIT_TYPE = 'Credit card' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_credit_card,
  SUM(CASE WHEN CREDIT_TYPE = 'Loan for purchase of shares (margin lending)' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_loan_for_purchase_of_shares_margin_lending,
  SUM(CASE WHEN CREDIT_TYPE = 'Loan for purchase of shares (margin lending)' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_loan_for_purchase_of_shares_margin_lending,
  SUM(CASE WHEN CREDIT_TYPE = 'Loan for purchase of shares (margin lending)' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_loan_for_purchase_of_shares_margin_lending,
  SUM(CASE WHEN CREDIT_TYPE = 'Loan for purchase of shares (margin lending)' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_loan_for_purchase_of_shares_margin_lending,
  SUM(CASE WHEN CREDIT_CURRENCY = 'currency 2' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_currency_currency_2,
  SUM(CASE WHEN CREDIT_CURRENCY = 'currency 2' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_currency_currency_2,
  SUM(CASE WHEN CREDIT_CURRENCY = 'currency 2' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_currency_currency_2,
  SUM(CASE WHEN CREDIT_CURRENCY = 'currency 2' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_currency_currency_2,
  SUM(CASE WHEN CREDIT_CURRENCY = 'currency 1' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_currency_currency_1,
  SUM(CASE WHEN CREDIT_CURRENCY = 'currency 1' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_currency_currency_1,
  SUM(CASE WHEN CREDIT_CURRENCY = 'currency 1' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_currency_currency_1,
  SUM(CASE WHEN CREDIT_CURRENCY = 'currency 1' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_currency_currency_1,
  SUM(CASE WHEN CREDIT_CURRENCY = 'currency 4' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_currency_currency_4,
  SUM(CASE WHEN CREDIT_CURRENCY = 'currency 4' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_currency_currency_4,
  SUM(CASE WHEN CREDIT_CURRENCY = 'currency 4' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_currency_currency_4,
  SUM(CASE WHEN CREDIT_CURRENCY = 'currency 4' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_currency_currency_4,
  SUM(CASE WHEN CREDIT_CURRENCY = 'currency 3' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_currency_currency_3,
  SUM(CASE WHEN CREDIT_CURRENCY = 'currency 3' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_currency_currency_3,
  SUM(CASE WHEN CREDIT_CURRENCY = 'currency 3' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_currency_currency_3,
  SUM(CASE WHEN CREDIT_CURRENCY = 'currency 3' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_currency_currency_3,
  MAX(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_active_bad_debt,
  MAX(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_active_bad_debt,
  MAX(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_active_bad_debt,
  MAX(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_active_bad_debt,
  MAX(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_active_sold,
  MAX(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_active_sold,
  MAX(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_active_sold,
  MAX(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_active_sold,
  MAX(CASE WHEN CREDIT_ACTIVE = 'Active' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_active_active,
  MAX(CASE WHEN CREDIT_ACTIVE = 'Active' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_active_active,
  MAX(CASE WHEN CREDIT_ACTIVE = 'Active' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_active_active,
  MAX(CASE WHEN CREDIT_ACTIVE = 'Active' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_active_active,
  MAX(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_active_closed,
  MAX(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_active_closed,
  MAX(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_active_closed,
  MAX(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_active_closed,
  MAX(CASE WHEN CREDIT_TYPE = 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_loan_for_the_purchase_of_equipment,
  MAX(CASE WHEN CREDIT_TYPE = 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_loan_for_the_purchase_of_equipment,
  MAX(CASE WHEN CREDIT_TYPE = 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_loan_for_the_purchase_of_equipment,
  MAX(CASE WHEN CREDIT_TYPE = 'Loan for the purchase of equipment' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_loan_for_the_purchase_of_equipment,
  MAX(CASE WHEN CREDIT_TYPE = 'Cash loan (non-earmarked)' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_cash_loan_non_earmarked,
  MAX(CASE WHEN CREDIT_TYPE = 'Cash loan (non-earmarked)' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_cash_loan_non_earmarked,
  MAX(CASE WHEN CREDIT_TYPE = 'Cash loan (non-earmarked)' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_cash_loan_non_earmarked,
  MAX(CASE WHEN CREDIT_TYPE = 'Cash loan (non-earmarked)' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_cash_loan_non_earmarked,
  MAX(CASE WHEN CREDIT_TYPE = 'Microloan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_microloan,
  MAX(CASE WHEN CREDIT_TYPE = 'Microloan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_microloan,
  MAX(CASE WHEN CREDIT_TYPE = 'Microloan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_microloan,
  MAX(CASE WHEN CREDIT_TYPE = 'Microloan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_microloan,
  MAX(CASE WHEN CREDIT_TYPE = 'Consumer credit' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_consumer_credit,
  MAX(CASE WHEN CREDIT_TYPE = 'Consumer credit' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_consumer_credit,
  MAX(CASE WHEN CREDIT_TYPE = 'Consumer credit' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_consumer_credit,
  MAX(CASE WHEN CREDIT_TYPE = 'Consumer credit' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_consumer_credit,
  MAX(CASE WHEN CREDIT_TYPE = 'Mobile operator loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_mobile_operator_loan,
  MAX(CASE WHEN CREDIT_TYPE = 'Mobile operator loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_mobile_operator_loan,
  MAX(CASE WHEN CREDIT_TYPE = 'Mobile operator loan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_mobile_operator_loan,
  MAX(CASE WHEN CREDIT_TYPE = 'Mobile operator loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_mobile_operator_loan,
  MAX(CASE WHEN CREDIT_TYPE = 'Another type of loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_another_type_of_loan,
  MAX(CASE WHEN CREDIT_TYPE = 'Another type of loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_another_type_of_loan,
  MAX(CASE WHEN CREDIT_TYPE = 'Another type of loan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_another_type_of_loan,
  MAX(CASE WHEN CREDIT_TYPE = 'Another type of loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_another_type_of_loan,
  MAX(CASE WHEN CREDIT_TYPE = 'Mortgage' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_mortgage,
  MAX(CASE WHEN CREDIT_TYPE = 'Mortgage' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_mortgage,
  MAX(CASE WHEN CREDIT_TYPE = 'Mortgage' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_mortgage,
  MAX(CASE WHEN CREDIT_TYPE = 'Mortgage' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_mortgage,
  MAX(CASE WHEN CREDIT_TYPE = 'Interbank credit' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_interbank_credit,
  MAX(CASE WHEN CREDIT_TYPE = 'Interbank credit' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_interbank_credit,
  MAX(CASE WHEN CREDIT_TYPE = 'Interbank credit' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_interbank_credit,
  MAX(CASE WHEN CREDIT_TYPE = 'Interbank credit' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_interbank_credit,
  MAX(CASE WHEN CREDIT_TYPE = 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_loan_for_working_capital_replenishment,
  MAX(CASE WHEN CREDIT_TYPE = 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_loan_for_working_capital_replenishment,
  MAX(CASE WHEN CREDIT_TYPE = 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_loan_for_working_capital_replenishment,
  MAX(CASE WHEN CREDIT_TYPE = 'Loan for working capital replenishment' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_loan_for_working_capital_replenishment,
  MAX(CASE WHEN CREDIT_TYPE = 'Car loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_car_loan,
  MAX(CASE WHEN CREDIT_TYPE = 'Car loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_car_loan,
  MAX(CASE WHEN CREDIT_TYPE = 'Car loan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_car_loan,
  MAX(CASE WHEN CREDIT_TYPE = 'Car loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_car_loan,
  MAX(CASE WHEN CREDIT_TYPE = 'Real estate loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_real_estate_loan,
  MAX(CASE WHEN CREDIT_TYPE = 'Real estate loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_real_estate_loan,
  MAX(CASE WHEN CREDIT_TYPE = 'Real estate loan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_real_estate_loan,
  MAX(CASE WHEN CREDIT_TYPE = 'Real estate loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_real_estate_loan,
  MAX(CASE WHEN CREDIT_TYPE = 'Unknown type of loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_unknown_type_of_loan,
  MAX(CASE WHEN CREDIT_TYPE = 'Unknown type of loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_unknown_type_of_loan,
  MAX(CASE WHEN CREDIT_TYPE = 'Unknown type of loan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_unknown_type_of_loan,
  MAX(CASE WHEN CREDIT_TYPE = 'Unknown type of loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_unknown_type_of_loan,
  MAX(CASE WHEN CREDIT_TYPE = 'Loan for business development' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_loan_for_business_development,
  MAX(CASE WHEN CREDIT_TYPE = 'Loan for business development' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_loan_for_business_development,
  MAX(CASE WHEN CREDIT_TYPE = 'Loan for business development' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_loan_for_business_development,
  MAX(CASE WHEN CREDIT_TYPE = 'Loan for business development' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_loan_for_business_development,
  MAX(CASE WHEN CREDIT_TYPE = 'Credit card' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_credit_card,
  MAX(CASE WHEN CREDIT_TYPE = 'Credit card' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_credit_card,
  MAX(CASE WHEN CREDIT_TYPE = 'Credit card' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_credit_card,
  MAX(CASE WHEN CREDIT_TYPE = 'Credit card' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_credit_card,
  MAX(CASE WHEN CREDIT_TYPE = 'Loan for purchase of shares (margin lending)' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_loan_for_purchase_of_shares_margin_lending,
  MAX(CASE WHEN CREDIT_TYPE = 'Loan for purchase of shares (margin lending)' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_loan_for_purchase_of_shares_margin_lending,
  MAX(CASE WHEN CREDIT_TYPE = 'Loan for purchase of shares (margin lending)' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_loan_for_purchase_of_shares_margin_lending,
  MAX(CASE WHEN CREDIT_TYPE = 'Loan for purchase of shares (margin lending)' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_loan_for_purchase_of_shares_margin_lending,
  MAX(CASE WHEN CREDIT_CURRENCY = 'currency 2' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_currency_currency_2,
  MAX(CASE WHEN CREDIT_CURRENCY = 'currency 2' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_currency_currency_2,
  MAX(CASE WHEN CREDIT_CURRENCY = 'currency 2' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_currency_currency_2,
  MAX(CASE WHEN CREDIT_CURRENCY = 'currency 2' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_currency_currency_2,
  MAX(CASE WHEN CREDIT_CURRENCY = 'currency 1' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_currency_currency_1,
  MAX(CASE WHEN CREDIT_CURRENCY = 'currency 1' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_currency_currency_1,
  MAX(CASE WHEN CREDIT_CURRENCY = 'currency 1' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_currency_currency_1,
  MAX(CASE WHEN CREDIT_CURRENCY = 'currency 1' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_currency_currency_1,
  MAX(CASE WHEN CREDIT_CURRENCY = 'currency 4' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_currency_currency_4,
  MAX(CASE WHEN CREDIT_CURRENCY = 'currency 4' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_currency_currency_4,
  MAX(CASE WHEN CREDIT_CURRENCY = 'currency 4' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_currency_currency_4,
  MAX(CASE WHEN CREDIT_CURRENCY = 'currency 4' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_currency_currency_4,
  MAX(CASE WHEN CREDIT_CURRENCY = 'currency 3' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_currency_currency_3,
  MAX(CASE WHEN CREDIT_CURRENCY = 'currency 3' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_currency_currency_3,
  MAX(CASE WHEN CREDIT_CURRENCY = 'currency 3' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_currency_currency_3,
  MAX(CASE WHEN CREDIT_CURRENCY = 'currency 3' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_currency_currency_3,
  MIN(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_active_bad_debt,
  MIN(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_active_bad_debt,
  MIN(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_active_bad_debt,
  MIN(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_active_bad_debt,
  MIN(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_active_sold,
  MIN(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_active_sold,
  MIN(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_active_sold,
  MIN(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_active_sold,
  MIN(CASE WHEN CREDIT_ACTIVE = 'Active' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_active_active,
  MIN(CASE WHEN CREDIT_ACTIVE = 'Active' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_active_active,
  MIN(CASE WHEN CREDIT_ACTIVE = 'Active' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_active_active,
  MIN(CASE WHEN CREDIT_ACTIVE = 'Active' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_active_active,
  MIN(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_active_closed,
  MIN(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_active_closed,
  MIN(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_active_closed,
  MIN(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_active_closed,
  MIN(CASE WHEN CREDIT_TYPE = 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_loan_for_the_purchase_of_equipment,
  MIN(CASE WHEN CREDIT_TYPE = 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_loan_for_the_purchase_of_equipment,
  MIN(CASE WHEN CREDIT_TYPE = 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_loan_for_the_purchase_of_equipment,
  MIN(CASE WHEN CREDIT_TYPE = 'Loan for the purchase of equipment' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_loan_for_the_purchase_of_equipment,
  MIN(CASE WHEN CREDIT_TYPE = 'Cash loan (non-earmarked)' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_cash_loan_non_earmarked,
  MIN(CASE WHEN CREDIT_TYPE = 'Cash loan (non-earmarked)' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_cash_loan_non_earmarked,
  MIN(CASE WHEN CREDIT_TYPE = 'Cash loan (non-earmarked)' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_cash_loan_non_earmarked,
  MIN(CASE WHEN CREDIT_TYPE = 'Cash loan (non-earmarked)' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_cash_loan_non_earmarked,
  MIN(CASE WHEN CREDIT_TYPE = 'Microloan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_microloan,
  MIN(CASE WHEN CREDIT_TYPE = 'Microloan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_microloan,
  MIN(CASE WHEN CREDIT_TYPE = 'Microloan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_microloan,
  MIN(CASE WHEN CREDIT_TYPE = 'Microloan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_microloan,
  MIN(CASE WHEN CREDIT_TYPE = 'Consumer credit' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_consumer_credit,
  MIN(CASE WHEN CREDIT_TYPE = 'Consumer credit' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_consumer_credit,
  MIN(CASE WHEN CREDIT_TYPE = 'Consumer credit' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_consumer_credit,
  MIN(CASE WHEN CREDIT_TYPE = 'Consumer credit' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_consumer_credit,
  MIN(CASE WHEN CREDIT_TYPE = 'Mobile operator loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_mobile_operator_loan,
  MIN(CASE WHEN CREDIT_TYPE = 'Mobile operator loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_mobile_operator_loan,
  MIN(CASE WHEN CREDIT_TYPE = 'Mobile operator loan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_mobile_operator_loan,
  MIN(CASE WHEN CREDIT_TYPE = 'Mobile operator loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_mobile_operator_loan,
  MIN(CASE WHEN CREDIT_TYPE = 'Another type of loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_another_type_of_loan,
  MIN(CASE WHEN CREDIT_TYPE = 'Another type of loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_another_type_of_loan,
  MIN(CASE WHEN CREDIT_TYPE = 'Another type of loan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_another_type_of_loan,
  MIN(CASE WHEN CREDIT_TYPE = 'Another type of loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_another_type_of_loan,
  MIN(CASE WHEN CREDIT_TYPE = 'Mortgage' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_mortgage,
  MIN(CASE WHEN CREDIT_TYPE = 'Mortgage' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_mortgage,
  MIN(CASE WHEN CREDIT_TYPE = 'Mortgage' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_mortgage,
  MIN(CASE WHEN CREDIT_TYPE = 'Mortgage' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_mortgage,
  MIN(CASE WHEN CREDIT_TYPE = 'Interbank credit' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_interbank_credit,
  MIN(CASE WHEN CREDIT_TYPE = 'Interbank credit' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_interbank_credit,
  MIN(CASE WHEN CREDIT_TYPE = 'Interbank credit' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_interbank_credit,
  MIN(CASE WHEN CREDIT_TYPE = 'Interbank credit' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_interbank_credit,
  MIN(CASE WHEN CREDIT_TYPE = 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_loan_for_working_capital_replenishment,
  MIN(CASE WHEN CREDIT_TYPE = 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_loan_for_working_capital_replenishment,
  MIN(CASE WHEN CREDIT_TYPE = 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_loan_for_working_capital_replenishment,
  MIN(CASE WHEN CREDIT_TYPE = 'Loan for working capital replenishment' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_loan_for_working_capital_replenishment,
  MIN(CASE WHEN CREDIT_TYPE = 'Car loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_car_loan,
  MIN(CASE WHEN CREDIT_TYPE = 'Car loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_car_loan,
  MIN(CASE WHEN CREDIT_TYPE = 'Car loan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_car_loan,
  MIN(CASE WHEN CREDIT_TYPE = 'Car loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_car_loan,
  MIN(CASE WHEN CREDIT_TYPE = 'Real estate loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_real_estate_loan,
  MIN(CASE WHEN CREDIT_TYPE = 'Real estate loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_real_estate_loan,
  MIN(CASE WHEN CREDIT_TYPE = 'Real estate loan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_real_estate_loan,
  MIN(CASE WHEN CREDIT_TYPE = 'Real estate loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_real_estate_loan,
  MIN(CASE WHEN CREDIT_TYPE = 'Unknown type of loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_unknown_type_of_loan,
  MIN(CASE WHEN CREDIT_TYPE = 'Unknown type of loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_unknown_type_of_loan,
  MIN(CASE WHEN CREDIT_TYPE = 'Unknown type of loan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_unknown_type_of_loan,
  MIN(CASE WHEN CREDIT_TYPE = 'Unknown type of loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_unknown_type_of_loan,
  MIN(CASE WHEN CREDIT_TYPE = 'Loan for business development' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_loan_for_business_development,
  MIN(CASE WHEN CREDIT_TYPE = 'Loan for business development' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_loan_for_business_development,
  MIN(CASE WHEN CREDIT_TYPE = 'Loan for business development' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_loan_for_business_development,
  MIN(CASE WHEN CREDIT_TYPE = 'Loan for business development' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_loan_for_business_development,
  MIN(CASE WHEN CREDIT_TYPE = 'Credit card' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_credit_card,
  MIN(CASE WHEN CREDIT_TYPE = 'Credit card' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_credit_card,
  MIN(CASE WHEN CREDIT_TYPE = 'Credit card' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_credit_card,
  MIN(CASE WHEN CREDIT_TYPE = 'Credit card' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_credit_card,
  MIN(CASE WHEN CREDIT_TYPE = 'Loan for purchase of shares (margin lending)' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_loan_for_purchase_of_shares_margin_lending,
  MIN(CASE WHEN CREDIT_TYPE = 'Loan for purchase of shares (margin lending)' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_loan_for_purchase_of_shares_margin_lending,
  MIN(CASE WHEN CREDIT_TYPE = 'Loan for purchase of shares (margin lending)' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_loan_for_purchase_of_shares_margin_lending,
  MIN(CASE WHEN CREDIT_TYPE = 'Loan for purchase of shares (margin lending)' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_loan_for_purchase_of_shares_margin_lending,
  MIN(CASE WHEN CREDIT_CURRENCY = 'currency 2' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_currency_currency_2,
  MIN(CASE WHEN CREDIT_CURRENCY = 'currency 2' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_currency_currency_2,
  MIN(CASE WHEN CREDIT_CURRENCY = 'currency 2' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_currency_currency_2,
  MIN(CASE WHEN CREDIT_CURRENCY = 'currency 2' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_currency_currency_2,
  MIN(CASE WHEN CREDIT_CURRENCY = 'currency 1' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_currency_currency_1,
  MIN(CASE WHEN CREDIT_CURRENCY = 'currency 1' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_currency_currency_1,
  MIN(CASE WHEN CREDIT_CURRENCY = 'currency 1' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_currency_currency_1,
  MIN(CASE WHEN CREDIT_CURRENCY = 'currency 1' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_currency_currency_1,
  MIN(CASE WHEN CREDIT_CURRENCY = 'currency 4' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_currency_currency_4,
  MIN(CASE WHEN CREDIT_CURRENCY = 'currency 4' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_currency_currency_4,
  MIN(CASE WHEN CREDIT_CURRENCY = 'currency 4' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_currency_currency_4,
  MIN(CASE WHEN CREDIT_CURRENCY = 'currency 4' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_currency_currency_4,
  MIN(CASE WHEN CREDIT_CURRENCY = 'currency 3' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_currency_currency_3,
  MIN(CASE WHEN CREDIT_CURRENCY = 'currency 3' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_currency_currency_3,
  MIN(CASE WHEN CREDIT_CURRENCY = 'currency 3' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_currency_currency_3,
  MIN(CASE WHEN CREDIT_CURRENCY = 'currency 3' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_currency_currency_3,
  AVG(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_active_bad_debt,
  AVG(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_active_bad_debt,
  AVG(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_active_bad_debt,
  AVG(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_active_bad_debt,
  AVG(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_active_sold,
  AVG(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_active_sold,
  AVG(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_active_sold,
  AVG(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_active_sold,
  AVG(CASE WHEN CREDIT_ACTIVE = 'Active' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_active_active,
  AVG(CASE WHEN CREDIT_ACTIVE = 'Active' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_active_active,
  AVG(CASE WHEN CREDIT_ACTIVE = 'Active' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_active_active,
  AVG(CASE WHEN CREDIT_ACTIVE = 'Active' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_active_active,
  AVG(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_active_closed,
  AVG(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_active_closed,
  AVG(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_active_closed,
  AVG(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_active_closed,
  AVG(CASE WHEN CREDIT_TYPE = 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_loan_for_the_purchase_of_equipment,
  AVG(CASE WHEN CREDIT_TYPE = 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_loan_for_the_purchase_of_equipment,
  AVG(CASE WHEN CREDIT_TYPE = 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_loan_for_the_purchase_of_equipment,
  AVG(CASE WHEN CREDIT_TYPE = 'Loan for the purchase of equipment' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_loan_for_the_purchase_of_equipment,
  AVG(CASE WHEN CREDIT_TYPE = 'Cash loan (non-earmarked)' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_cash_loan_non_earmarked,
  AVG(CASE WHEN CREDIT_TYPE = 'Cash loan (non-earmarked)' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_cash_loan_non_earmarked,
  AVG(CASE WHEN CREDIT_TYPE = 'Cash loan (non-earmarked)' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_cash_loan_non_earmarked,
  AVG(CASE WHEN CREDIT_TYPE = 'Cash loan (non-earmarked)' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_cash_loan_non_earmarked,
  AVG(CASE WHEN CREDIT_TYPE = 'Microloan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_microloan,
  AVG(CASE WHEN CREDIT_TYPE = 'Microloan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_microloan,
  AVG(CASE WHEN CREDIT_TYPE = 'Microloan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_microloan,
  AVG(CASE WHEN CREDIT_TYPE = 'Microloan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_microloan,
  AVG(CASE WHEN CREDIT_TYPE = 'Consumer credit' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_consumer_credit,
  AVG(CASE WHEN CREDIT_TYPE = 'Consumer credit' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_consumer_credit,
  AVG(CASE WHEN CREDIT_TYPE = 'Consumer credit' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_consumer_credit,
  AVG(CASE WHEN CREDIT_TYPE = 'Consumer credit' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_consumer_credit,
  AVG(CASE WHEN CREDIT_TYPE = 'Mobile operator loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_mobile_operator_loan,
  AVG(CASE WHEN CREDIT_TYPE = 'Mobile operator loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_mobile_operator_loan,
  AVG(CASE WHEN CREDIT_TYPE = 'Mobile operator loan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_mobile_operator_loan,
  AVG(CASE WHEN CREDIT_TYPE = 'Mobile operator loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_mobile_operator_loan,
  AVG(CASE WHEN CREDIT_TYPE = 'Another type of loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_another_type_of_loan,
  AVG(CASE WHEN CREDIT_TYPE = 'Another type of loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_another_type_of_loan,
  AVG(CASE WHEN CREDIT_TYPE = 'Another type of loan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_another_type_of_loan,
  AVG(CASE WHEN CREDIT_TYPE = 'Another type of loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_another_type_of_loan,
  AVG(CASE WHEN CREDIT_TYPE = 'Mortgage' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_mortgage,
  AVG(CASE WHEN CREDIT_TYPE = 'Mortgage' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_mortgage,
  AVG(CASE WHEN CREDIT_TYPE = 'Mortgage' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_mortgage,
  AVG(CASE WHEN CREDIT_TYPE = 'Mortgage' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_mortgage,
  AVG(CASE WHEN CREDIT_TYPE = 'Interbank credit' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_interbank_credit,
  AVG(CASE WHEN CREDIT_TYPE = 'Interbank credit' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_interbank_credit,
  AVG(CASE WHEN CREDIT_TYPE = 'Interbank credit' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_interbank_credit,
  AVG(CASE WHEN CREDIT_TYPE = 'Interbank credit' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_interbank_credit,
  AVG(CASE WHEN CREDIT_TYPE = 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_loan_for_working_capital_replenishment,
  AVG(CASE WHEN CREDIT_TYPE = 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_loan_for_working_capital_replenishment,
  AVG(CASE WHEN CREDIT_TYPE = 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_loan_for_working_capital_replenishment,
  AVG(CASE WHEN CREDIT_TYPE = 'Loan for working capital replenishment' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_loan_for_working_capital_replenishment,
  AVG(CASE WHEN CREDIT_TYPE = 'Car loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_car_loan,
  AVG(CASE WHEN CREDIT_TYPE = 'Car loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_car_loan,
  AVG(CASE WHEN CREDIT_TYPE = 'Car loan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_car_loan,
  AVG(CASE WHEN CREDIT_TYPE = 'Car loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_car_loan,
  AVG(CASE WHEN CREDIT_TYPE = 'Real estate loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_real_estate_loan,
  AVG(CASE WHEN CREDIT_TYPE = 'Real estate loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_real_estate_loan,
  AVG(CASE WHEN CREDIT_TYPE = 'Real estate loan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_real_estate_loan,
  AVG(CASE WHEN CREDIT_TYPE = 'Real estate loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_real_estate_loan,
  AVG(CASE WHEN CREDIT_TYPE = 'Unknown type of loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_unknown_type_of_loan,
  AVG(CASE WHEN CREDIT_TYPE = 'Unknown type of loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_unknown_type_of_loan,
  AVG(CASE WHEN CREDIT_TYPE = 'Unknown type of loan' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_unknown_type_of_loan,
  AVG(CASE WHEN CREDIT_TYPE = 'Unknown type of loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_unknown_type_of_loan,
  AVG(CASE WHEN CREDIT_TYPE = 'Loan for business development' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_loan_for_business_development,
  AVG(CASE WHEN CREDIT_TYPE = 'Loan for business development' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_loan_for_business_development,
  AVG(CASE WHEN CREDIT_TYPE = 'Loan for business development' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_loan_for_business_development,
  AVG(CASE WHEN CREDIT_TYPE = 'Loan for business development' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_loan_for_business_development,
  AVG(CASE WHEN CREDIT_TYPE = 'Credit card' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_credit_card,
  AVG(CASE WHEN CREDIT_TYPE = 'Credit card' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_credit_card,
  AVG(CASE WHEN CREDIT_TYPE = 'Credit card' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_credit_card,
  AVG(CASE WHEN CREDIT_TYPE = 'Credit card' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_credit_card,
  AVG(CASE WHEN CREDIT_TYPE = 'Loan for purchase of shares (margin lending)' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_type_loan_for_purchase_of_shares_margin_lending,
  AVG(CASE WHEN CREDIT_TYPE = 'Loan for purchase of shares (margin lending)' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_type_loan_for_purchase_of_shares_margin_lending,
  AVG(CASE WHEN CREDIT_TYPE = 'Loan for purchase of shares (margin lending)' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_type_loan_for_purchase_of_shares_margin_lending,
  AVG(CASE WHEN CREDIT_TYPE = 'Loan for purchase of shares (margin lending)' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_type_loan_for_purchase_of_shares_margin_lending,
  AVG(CASE WHEN CREDIT_CURRENCY = 'currency 2' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_currency_currency_2,
  AVG(CASE WHEN CREDIT_CURRENCY = 'currency 2' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_currency_currency_2,
  AVG(CASE WHEN CREDIT_CURRENCY = 'currency 2' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_currency_currency_2,
  AVG(CASE WHEN CREDIT_CURRENCY = 'currency 2' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_currency_currency_2,
  AVG(CASE WHEN CREDIT_CURRENCY = 'currency 1' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_currency_currency_1,
  AVG(CASE WHEN CREDIT_CURRENCY = 'currency 1' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_currency_currency_1,
  AVG(CASE WHEN CREDIT_CURRENCY = 'currency 1' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_currency_currency_1,
  AVG(CASE WHEN CREDIT_CURRENCY = 'currency 1' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_currency_currency_1,
  AVG(CASE WHEN CREDIT_CURRENCY = 'currency 4' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_currency_currency_4,
  AVG(CASE WHEN CREDIT_CURRENCY = 'currency 4' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_currency_currency_4,
  AVG(CASE WHEN CREDIT_CURRENCY = 'currency 4' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_currency_currency_4,
  AVG(CASE WHEN CREDIT_CURRENCY = 'currency 4' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_currency_currency_4,
  AVG(CASE WHEN CREDIT_CURRENCY = 'currency 3' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) as amt_credit_sum_limit_credit_currency_currency_3,
  AVG(CASE WHEN CREDIT_CURRENCY = 'currency 3' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) as amt_credit_sum_debt_credit_currency_currency_3,
  AVG(CASE WHEN CREDIT_CURRENCY = 'currency 3' THEN AMT_CREDIT_SUM ELSE 0 END) as amt_credit_sum_credit_currency_currency_3,
  AVG(CASE WHEN CREDIT_CURRENCY = 'currency 3' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) as amt_credit_max_overdue_credit_currency_currency_3
FROM
  df_view
GROUP BY
  SK_ID_CURR
''')
df_bureau_006.show()

+----------+-------------------------------------------+------------------------------------------+-------------------------------------+---------------------------------------------+---------------------------------------+--------------------------------------+---------------------------------+-----------------------------------------+-----------------------------------------+----------------------------------------+-----------------------------------+-------------------------------------------+-----------------------------------------+----------------------------------------+-----------------------------------+-------------------------------------------+-------------------------------------------------------------------+------------------------------------------------------------------+-------------------------------------------------------------+---------------------------------------------------------------------+--------------------------------------------------------+------------

### **5.4 - Quantidade de variáveis**

In [ ]:
print(len(df_bureau_006.columns))

369


## **6 - Conclusão**

Por fim, é possível verificar que a partir das práticas realizadas, a nossa tabela de treinamento agora conta com 369 variáveis preditivas a mais do que inicialmente.

Essas variáveis criadas carregam informações relevantes sobre os históricos de transações dos indivíduos, o que é de extrema importancia na excecução de modelos estatisticos que podem ser realizados posteriormente para ajudar instituições financeiras a resolverem problemas de credito, dentre outras questões.

As variáveis criadas foram apenas exemplos do que podem ser feitas com tabelas transacionais. Além dessas variáveis, também é possível obter informações históricas dos individuos por categoria de produto, como por exemplo, quanto um individuo gastou no mercado nos ultimos 3, 6 e 9 meses. Esses são outros exemplos de como podemos enriquecer ainda mais a base de dados - sendo possível criar milhares de variáveis preditivas.

Por fim, notamos um acréscimo de 369 variáveis preditivas à nossa tabela de treinamento em comparação à tabela inicial. Essas novas variáveis nos mostram informações relevantes sobre os históricos de transações dos indivíduos, desempenhando um papel crucial na execução de modelos estatísticos pelos Cientistas de Dados. Tais modelos que poderão ser empregados para auxiliar instituições financeiras na resolução de problemas como: avaliação de risco de inadimplência, determinação de limites de crédito, detecção de fraude, dentre outras questões.

As variáveis criadas representam apenas uma amostra do potencial das tabelas transacionais. Além dessas, também é possível extrair informações históricas dos indivíduos, segmentadas por categoria de produto. Por exemplo, podemos calcular os gastos de um indivíduo no mercado nos últimos 3, 6 e 9 meses. Estes são apenas alguns exemplos que demonstram como é possível enriquecer ainda mais uma base de dados, abrindo possibilidades para a criação de milhares de variáveis preditivas.